In [1]:
from typing import Dict, Union

from IPython import get_ipython
from IPython.display import display, Image
import csv

In [2]:
import autogen, os
import json
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

cache_seed = 41 # default seed is 41


my_models = [
    {
        'model': os.environ['GPT4-1106-128k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    },
    {
        'model': os.environ['GPT432-0613-32k'],
        'api_key': os.environ['AZURE_OPENAI_API_KEY'],
        "base_url": os.environ['AZURE_OPENAI_ENDPOINT'],
        "api_type": os.environ['OPENAI_API_TYPE'],
        "api_version": os.environ['AZURE_OPENAI_API_VERSION']
    }
]

# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "model": {
            "gpt4-1106-128k",
            "gpt4-0613-32k"
        }
    }
)

config_list

[{'model': 'gpt4-1106-128k',
  'api_key': '29eb6e832af54fec9e19e76d185e4557',
  'base_url': 'https://mmopenaisec.openai.azure.com/',
  'api_type': 'azure',
  'api_version': '2024-02-15-preview'},
 {'model': 'gpt4-0613-32k',
  'api_key': '29eb6e832af54fec9e19e76d185e4557',
  'base_url': 'https://mmopenaisec.openai.azure.com/',
  'api_type': 'azure',
  'api_version': '2024-02-15-preview'}]

In [3]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    }
)

user_proxy

In [4]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "cache_seed": cache_seed,  # seed for caching and reproducibility
        "config_list": config_list,  # a list of OpenAI API configurations
        "temperature": 0,  # temperature for sampling
    },  # configuration for autogen's enhanced inference API which is compatible with OpenAI API
)

assistant

In [5]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description


chat_res = user_proxy.initiate_chat(
    assistant,
    message="""What date is today? Compare the year-to-date gain for META and TESLA.""",
    summary_method="reflection_with_llm",
)

user_proxy (to assistant):

What date is today? Compare the year-to-date gain for META and TESLA.

--------------------------------------------------------------------------------
assistant (to user_proxy):

To get the current date, we can use Python code to print it out. Then, to compare the year-to-date (YTD) gain for META (Meta Platforms, Inc.) and TESLA (Tesla, Inc.), we will need to retrieve their stock prices from the beginning of the year and their current stock prices. We can use financial APIs or web scraping to get this information, but since I cannot browse the web or use external APIs, I will provide you with Python code to get the current date first.

Please execute the following Python code to get the current date:

```python
# filename: get_current_date.py
import datetime

# Get the current date
current_date = datetime.date.today()

# Print the current date
print(f"Today's date is: {current_date}")
```

After you have the current date, please provide me with the stock pr

In [6]:
# autogen.Completion.clear_cache(seed=cache_seed)

In [7]:
i=1
for cr in chat_res.chat_history:
    print(i, cr["content"].rstrip()[-9:])
    i=i+1

1 nd TESLA.
2 his data.
3 024-03-27
4 ompanies.
5 
6  with me.
7 
8 ary data.
9 e unknown
10  me know.
11 
12 TERMINATE
